## Imports

In [2]:
import numpy as np
import pandas as pd
from ape import accounts, chain, Contract, networks, project

/Users/anantdeepparihar/Library/Application Support/hatch/env/virtual/kodiak-simulations-2023-07/MXHRkJMx/kodiak-simulations-2023-07/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

## Contract objects

In [4]:
# Load Uniswap factory contract
factory = Contract("0x1F98431c8aD98523631AE4a59f267346ea31F984")

# Load a referece pool; obtain `contract_type` from here to read any uniswap pool with ape
pool_ref = Contract('0x8f8EF111B67C04Eb1641f5ff19EE54Cda062f163')

# USDC/ETH 0.05%
pool = Contract("0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640", contract_type=pool_ref.contract_type)

token0 = Contract(pool.token0())
token1 = Contract(pool.token1())

## Get prices

#### Inputs for querying and saving price data

In [5]:
# start_block = 12376729  # ETH/USDC 0.05% pool deployment block
start_block = chain.blocks.head.number - 100_000
head_block = chain.blocks.head.number
step = 2400
filename = f"data/price_{pool.address}_{start_block}_{head_block}_{step}.csv"

#### Helpful functions

In [6]:
# calculate prices for both y/x and x/y in case of "weird" token0, token1 ordering
def price(sqrt_price_x96) -> int:
    return (int(sqrt_price_x96) >> 96) ** 2

#### Query prices

In [7]:
# query for tick data each block and insert into pandas dataframe
is_head = True
for block in range(start_block, head_block, step):
    print(f"Processing block {block} ...", end='\r')
    
    # get the sqrt price data at block
    slot0 = pool.slot0(block_identifier=block)
    y_by_x = price(slot0.sqrtPriceX96)
    row = {
        'block_number': [block],
        'timestamp': [chain.blocks[block].timestamp],
        'sqrt_price_x96': [slot0.sqrtPriceX96],
        'y/x': [y_by_x],
        'x/y': 10 ** token1.decimals() // y_by_x
    }
        
    # convert to pd dataframe then append to file
    df = pd.DataFrame(data=row)
    df.to_csv(filename, mode='a', index=False, header=is_head)
        
    if is_head:
        is_head = False

In [8]:
df = pd.read_csv(filename)
df.head()

,block_number,timestamp,sqrt_price_x96,price
0,17846003,1691205083,1853108454851453304155515201922274,0.000547
1,17848403,1691234099,1851322486752172637560000266968502,0.000546
2,17850803,1691263055,1849161700064716265427670227259749,0.000545
3,17853203,1691292083,1850319883285144859934317017619672,0.000545
4,17855603,1691321123,1852406778307200559820017252423304,0.000547


In [9]:
df.shape

(42, 4)